## Example submission

Image Matching Challenge 2025: https://www.kaggle.com/competitions/image-matching-challenge-2025

This notebook creates a simple submission using ALIKED and LightGlue, plus DINO for shortlisting, on GPU. Adapted from [last year](https://www.kaggle.com/code/oldufo/imc-2024-submission-example).

Remember to select an accelerator on the sidebar to the right, and to disable internet access when submitting a notebook to the competition.

In [1]:
# IMPORTANT 
#Install dependencies and copy model weights to run the notebook without internet access when submitting to the competition.

!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!pip install --no-index /kaggle/input/imc2025/* --no-deps

!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/aliked-n16.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
  Attempting uninstall: kornia-rs
    Found existing installation: kornia_rs 0.1.8
    Uninstalling kornia_rs-0.1.8:
      Successfully uninstalled kornia_rs-0.1.8
  Attempting uninstall: kornia
    Found existing installation: kornia 0.8.0
    Uninstalling kornia-0.8.0:
      Successfully uninstalled ko

In [2]:
import sys
import os
from tqdm import tqdm
from time import time, sleep
import gc
import numpy as np
import h5py
import dataclasses
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from PIL import Image

import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF

import torch
from lightglue import match_pair
from lightglue import ALIKED, LightGlue
from lightglue.utils import load_image, rbd
from transformers import AutoImageProcessor, AutoModel

# IMPORTANT Utilities: importing data into colmap and competition metric
import pycolmap
sys.path.append('/kaggle/input/imc25-utils')
from database import *
from h5_to_db import *
import metric
from torch.backends import cudnn


/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.10/dist-packages/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [3]:

# print("PyTorch version:", torch.__version__)
# import sys
# print("Python version:", sys.version)

# print("CUDA available:", torch.cuda.is_available())
# print("CUDA version:", torch.version.cuda)
# print("Device count:", torch.cuda.device_count())
# print("Current device:", torch.cuda.current_device())
# print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


In [4]:
import importlib
import shutil


In [5]:
# Do not forget to select an accelerator on the sidebar to the right.
device = K.utils.get_cuda_device_if_available(0)
print(f'{device=}')

device=device(type='cpu')


In [6]:
# !zip -r /kaggle/working/result/featureout/ETs/featurept.zip /kaggle/working/result/featureout/ETs/featurept


In [7]:

data_dir = '/kaggle/input/image-matching-challenge-2025'
workdir = '/kaggle/working/result/'
os.makedirs(workdir, exist_ok=True)

In [8]:
from pathlib import Path

def draw_and_save_feature_points(image_path, keypoints, result_folder):
    """
    Draw feature points on the image and save to result folder.

    Args:
        image_path (str or Path): Path to the input image.
        keypoints (np.ndarray): (N, 2) array of (x, y) coordinates.
        result_folder (str or Path): Folder to save the output image.
    """
    # Load image in BGR
    return
    image = cv2.imread(str(image_path))
    if image is None:
        raise ValueError(f"Cannot read image from {image_path}")

    # Draw keypoints
    for (x, y) in keypoints.astype(int):
        cv2.circle(image, (x, y), radius=2, color=(0, 255, 0), thickness=-1)  # Green dots

    img_fname = image_path.split('/')[-1]

    result_folder = Path(result_folder)
    img_fname = Path(image_path).stem  # no extension
    output_path = result_folder / f"{img_fname}_fe.png"

    cv2.imwrite(str(output_path), image)
    print(f"Saved: {output_path}")

In [9]:
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

def gem(x, p=3, eps=1e-6):
    return F.normalize(torch.mean(x.clamp(min=eps).pow(p), dim=1).pow(1/p), p=2, dim=1)



# Must Use efficientnet global descriptor to get matching shortlists.
def get_global_desc(fnames, device = torch.device('cpu'), is_max = True):
    processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = model.eval()
    model = model.to(device)
    global_descs_dinov2 = []
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        timg = load_torch_image(img_fname_full)
        with torch.inference_mode():
            inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            if is_max:
                dino_fea = gem(outputs.last_hidden_state[:,1:], p=3)
            else:
                dino_fea = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=1, p=2)
        global_descs_dinov2.append(dino_fea.detach().cpu())
    global_descs_dinov2 = torch.cat(global_descs_dinov2, dim=0)
    return global_descs_dinov2


def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs


def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6, # should be strict
                              min_pairs = 10,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu'),
                              max_pairs = 30):
    num_imgs = len(fnames)
    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames)
    descs = get_global_desc(fnames, device=device, is_max = False)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()
    # print(dm)
    # 只分析上三角（去掉对角线），避免重复
    triu_indices = np.triu_indices_from(dm, k=1)
    dm_flat = dm[triu_indices]
    
    # 打印统计信息
    print("Distance Matrix Statistics:")
    print(f"Min:  {dm_flat.min():.4f}")
    print(f"Max:  {dm_flat.max():.4f}")
    print(f"Mean: {dm_flat.mean():.4f}")
    print(f"Std:  {dm_flat.std():.4f}")
    print(f"20%:  {np.percentile(dm_flat, 20):.4f}")
    print(f"30%:  {np.percentile(dm_flat, 30):.4f}")
    print(f"USED 50%:  {np.percentile(dm_flat, 50):.4f}")
    print(f"75%:  {np.percentile(dm_flat, 75):.4f}")
    
    thr = np.percentile(dm_flat, 50)
    threshold = max(dm_flat.mean() + np.sqrt(3) * dm_flat.std(), thr)
    # removing half
    mask = dm <= thr
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs :
            to_match = np.argsort(dm[st_idx])[:min_pairs]  
        if len(to_match) >= max_pairs:
            to_match = np.argsort(dm[st_idx])[:max_pairs]  
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < threshold:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

def detect_aliked(img_fnames,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 2048,
                  device=torch.device('cpu')):
    dtype = torch.float32 # ALIKED has issues with float16
    extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.1).eval().to(device, dtype)
    extractor.preprocess_conf["resize"] = resize_to
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)

    # Calculate the expected scale factor ALIKED will apply
    # ALIKED uses preprocess_conf["resize"] on the *input image tensor*
    # Input image tensor size will be (H, W) after Kornia loading/conversion
    
    draw_feature_dir = os.path.join(feature_dir, 'featurept')
    os.makedirs(draw_feature_dir, exist_ok=True)
    
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='w') as f_desc:
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = extractor.extract(image0)  # auto-resize the image, disable with resize=None
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
                draw_and_save_feature_points(img_path, kpts, draw_feature_dir)
    return

def match_with_lightglue(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=30,
                   verbose=False,
                   match_score_thresh = 0.15):
    lg_matcher = KF.LightGlueMatcher("aliked", {"width_confidence": -1,
                                                "depth_confidence": -1,
                                                 "mp": True if 'cuda' in str(device) else False}).eval().to(device)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='r') as f_kp, \
        h5py.File(f'{feature_dir}/descriptors.h5', mode='r') as f_desc, \
        h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for pair_idx in tqdm(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                         desc2,
                                         KF.laf_from_center_scale_ori(kp1[None]),
                                         KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0:
                continue
            n_matches = len(idxs)
            # if verbose:
            #     print (f'{key1}-{key2}: {n_matches} matches')
            # group  = f_match.require_group(key1)
            # if n_matches >= min_matches:
            #      group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
            # Filter by match score (distance)
            
            mask = dists > match_score_thresh
            idxs_filtered = idxs[mask.squeeze(1)]
    
            n_matches = len(idxs_filtered)
            if n_matches == 0:
                continue
    
            if verbose:
                print(f'{key1}-{key2}: {n_matches} matches (filtered from {len(idxs)})')
    
            group = f_match.require_group(key1)
            if n_matches >= min_matches:
                group.create_dataset(key2, data=idxs_filtered.detach().cpu().numpy().reshape(-1, 2))

    return



def import_into_colmap(img_dir, feature_dir ='.featureout', database_path = 'colmap.db'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, '', 'simple-pinhole', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()
    return

In [10]:
class Config:
    base_path = Path("/kaggle/input/image-matching-challenge-2025")
    feature_dir = Path(workdir) / ".feature_outputs"

    device = K.utils.get_cuda_device_if_available(0)
    num_gpus = torch.cuda.device_count()

    num_trials = 2

    # keypoint_detection_args = {
    #     "num_features": 4096,
    #     "detection_threshold": 0.01,
    #     "resize_to": 1024,
    # }

    # keypoint_distances_args = {
    #     "early_stopping_thr": 1000,
    #     "min_matches": [100, 125],
    #     "verbose": False,
    # }

    doppelgangers_classifier_args = {
        "loftr_weight_path": Path("/kaggle/input/loftr/pytorch/outdoor/1/loftr_outdoor.ckpt"),
        "doppelgangers_weight_path": Path("/kaggle/input/doppelgangers-repo/doppelgangers/weights/doppelgangers_classifier_loftr.pt"),
    }

    # remove_ambiguous_area_args = {
    #     "pcd_used_num_images_ratio": 0.5,
    #     "erode_mask_ratio": 0.0,
    # }

    verify_matches_args = {
        "doppelgangers_min_thr": 0.3,
        "doppelgangers_max_thr": 0.5,
        "filter_iterations": 1,
        "filter_threshold": 1,
    }

    # colmap_mapper_options = {
    #     "min_model_size": 3,
    #     "max_num_models": 2,
    # }

In [11]:
from concurrent.futures import ThreadPoolExecutor

sys.path.append("/kaggle/input/doppelgangers-repo/doppelgangers")
from doppelgangers.third_party.loftr import LoFTR, default_cfg
from doppelgangers.utils.loftr_matches import read_image

In [12]:
def extract_matching_list_from_h5(pair_probability_file_path, fname_to_index):
    """
    Convert h5 pair-probability file into list of index pairs.
    """
    # map: filename -> index    
    
    matching_list = []

    with h5py.File(pair_probability_file_path, 'r') as f:
        for key1 in f.keys():
            if key1 not in fname_to_index:
                continue
            for key2 in f[key1].keys():
                if key2 not in fname_to_index:
                    continue
                idx1, idx2 = fname_to_index[key1], fname_to_index[key2]
                if idx1 != idx2:
                    matching_list.append(tuple(sorted((idx1, idx2))))

    matching_list = sorted(list(set(matching_list)))  # remove duplicates
    return matching_list


In [13]:
import itertools
def save_loftr_matches(data_path, pairs_info, pairs_info_index, output_path, device_id, model_weight_path="weights/outdoor_ds.ckpt"):
    # The default config uses dual-softmax.
    # The outdoor and indoor models share the same config.
    # You can change the default values like thr and coarse_match_type.

    device = torch.device(f"cuda:{device_id}")

    matcher = LoFTR(config=default_cfg)
    matcher.load_state_dict(torch.load(model_weight_path, weights_only=False)['state_dict'])
    matcher = matcher.eval().to(device)

    img_size = 1024
    df = 8
    padding = True
    print("generating LOFTR")
    for _pairs_info, idx in tqdm(zip(pairs_info, pairs_info_index), total=len(pairs_info_index), desc=f"Running LOFTR [GPU{device_id}]"):

        output_file_path = output_path / "loftr_match" / f"{idx}.npy"
        if output_file_path.exists():
            continue

        name0, name1, _, _, _ = _pairs_info

        img0_pth = data_path / name0
        img1_pth = data_path / name1
        img0_raw, mask0 = read_image(str(img0_pth), img_size, df, padding)
        img1_raw, mask1 = read_image(str(img1_pth), img_size, df, padding)
        img0 = torch.from_numpy(img0_raw).to(device)
        img1 = torch.from_numpy(img1_raw).to(device)
        mask0 = torch.from_numpy(mask0).to(device)
        mask1 = torch.from_numpy(mask1).to(device)
        batch = {'image0': img0, 'image1': img1, 'mask0': mask0, 'mask1':mask1}

        # Inference with LoFTR and get prediction
        
        with torch.no_grad():
            matcher(batch)
            mkpts0 = batch['mkpts0_f'].cpu().numpy()
            mkpts1 = batch['mkpts1_f'].cpu().numpy()
            mconf = batch['mconf'].cpu().numpy()
            np.save(output_file_path, {"kpt0": mkpts0, "kpt1": mkpts1, "conf": mconf})


def doppelgangers_classifier(cfg, pretrained_model_path, pair_path, drop_score = 0.2):
    # basic setup
    cudnn.benchmark = True

    # initial dataset
    data_lib = importlib.import_module(cfg.data.type)
    loaders = data_lib.get_data_loaders(cfg.data)
    test_loader = loaders["test_loader"]

    # initial model
    decoder_lib = importlib.import_module(cfg.models.decoder.type)
    decoder = decoder_lib.decoder(cfg.models.decoder)
    decoder = decoder.cuda()

    # load pretrained model
    ckpt = torch.load(pretrained_model_path, weights_only=False)
    new_ckpt = deepcopy(ckpt["dec"])
    for key, _ in ckpt["dec"].items():
        if "module." in key:
            new_ckpt[key[len("module."):]] = new_ckpt.pop(key)
    decoder.load_state_dict(new_ckpt, strict=True)

    # evaluate on test set
    decoder.eval()
    prob_list = list()
    with torch.no_grad():
        for _, data in tqdm(enumerate(test_loader), total=len(test_loader), desc="Running doppelgangers"):

            # TTA
            img = data["image"].cuda()

            img_list = []
            img_list.append(img)
            # lrflip
            # img = torch.flip(img, [3])
            # img_list.append(img)
            # stack
            img = torch.cat(img_list)

            logits = decoder(img)
            prob = torch.nn.functional.softmax(logits, dim=1)
            prob = torch.mean(prob, dim=0)
            prob_list.append(prob[1].detach().cpu().numpy())

    y_scores = np.array(prob_list)

    pairs_info = np.load(pair_path, allow_pickle=True)
    pair_probability_file_path = cfg.data.output_path / "pair_probability_list.h5"
    with h5py.File(pair_probability_file_path, mode="w") as f_matches:

        for idx in range(pairs_info.shape[0]):

            key1, key2, _, _, _ = pairs_info[idx]
            score = y_scores[idx]
            if(score > drop_score):
                group  = f_matches.require_group(key1)
                group.create_dataset(key2, data=score)

    return pair_probability_file_path

def create_image_pair_list(matches_path, output_path):

    dummy = 0
    pairs_list = []
    with h5py.File(matches_path, mode="r") as f_matches:

        for key1 in f_matches.keys():
            group = f_matches[key1]
            for key2 in group.keys():
                pairs_list.append([key1, key2, dummy, dummy, dummy])

    pairs_list = np.concatenate(pairs_list, axis=0).reshape(-1, 5)

    pairs_list_path = output_path / "pairs_list.npy"
    np.save(pairs_list_path, pairs_list)

    return pairs_list_path

def create_image_pair_list_frompairs(images, index_pairs, output_path):

    dummy = 0
    pairs_list = []
    
    for i, pair in enumerate(index_pairs):
        key1 = Path(images[pair[0]]).name
        key2 = Path(images[pair[1]]).name
        pairs_list.append([key1, key2, dummy, dummy, dummy])
    pairs_list = np.concatenate(pairs_list, axis=0).reshape(-1, 5)
    # print(pairs_list)
    pairs_list_path = output_path / "pairs_list.npy"
    np.save(pairs_list_path, pairs_list)

    return pairs_list_path

def exec_doppelgangers_classifier(
    images_dir,
    images,
    feature_dir,
    index_pairs,
    num_gpus,
    loftr_weight_path,
    doppelgangers_weight_path,
):
    print(feature_dir)
    loftr_matches_path = feature_dir / "loftr_match"
    loftr_matches_path.mkdir(parents=True, exist_ok=True)
    pair_path = create_image_pair_list_frompairs(images, index_pairs, feature_dir)
    pairs_info = np.load(pair_path, allow_pickle=True)
    pairs_info_index = np.arange(pairs_info.shape[0])
    
    with ThreadPoolExecutor() as executor:
        executor.map(
            save_loftr_matches,
            itertools.repeat(images_dir),
            np.array_split(pairs_info, num_gpus),
            np.array_split(pairs_info_index, num_gpus),
            itertools.repeat(feature_dir),
            range(num_gpus),
            itertools.repeat(loftr_weight_path),
        )
    gc.collect()
    torch.cuda.empty_cache()

    config = {
        "data": {
            "image_dir": images_dir,
            "loftr_match_dir": loftr_matches_path,
            "output_path": feature_dir,
            "type": "doppelgangers.datasets.sfm_disambiguation_dataset",
            "num_workers": 1,
            "test": {
                "batch_size": 1,
                "img_size": 1024,
                "pair_path": pair_path,
            },
        },
        "models": {
            "decoder": {
                "type": "doppelgangers.models.cnn_classifier",
                "input_dim": 10,
            },
        },
    }

    def dict2namespace(config):
        namespace = argparse.Namespace()
        for key, value in config.items():
            if isinstance(value, dict):
                new_value = dict2namespace(value)
            else:
                new_value = value
            setattr(namespace, key, new_value)
        return namespace
    
    config = dict2namespace(config)

    # Running Doppelgangers classifier model on image pairs
    print("Running Doppelgangers classifier model on image pairs")
    pair_probability_file_path = doppelgangers_classifier(config, doppelgangers_weight_path, pair_path)
    
    shutil.rmtree(loftr_matches_path)

    return pair_probability_file_path

In [14]:
# Collect vital info from the dataset

@dataclasses.dataclass
class Prediction:
    image_id: str | None  # A unique identifier for the row -- unused otherwise. Used only on the hidden test set.
    dataset: str
    filename: str
    cluster_index: int | None = None
    rotation: np.ndarray | None = None
    translation: np.ndarray | None = None

# Set is_train=True to run the notebook on the training data.
# Set is_train=False if submitting an entry to the competition (test data is hidden, and different from what you see on the "test" folder).
is_train = True

if is_train:
    sample_submission_csv = os.path.join(data_dir, 'train_labels.csv')
else:
    sample_submission_csv = os.path.join(data_dir, 'sample_submission.csv')

samples = {}
competition_data = pd.read_csv(sample_submission_csv)
for _, row in competition_data.iterrows():
    # Note: For the test data, the "scene" column has no meaning, and the rotation_matrix and translation_vector columns are random.
    if row.dataset not in samples:
        samples[row.dataset] = []
    samples[row.dataset].append(
        Prediction(
            image_id=None if is_train else row.image_id,
            dataset=row.dataset,
            filename=row.image
        )
    )

for dataset in samples:
    print(f'Dataset "{dataset}" -> num_images={len(samples[dataset])}')

Dataset "imc2023_haiper" -> num_images=54
Dataset "imc2023_heritage" -> num_images=209
Dataset "imc2023_theather_imc2024_church" -> num_images=76
Dataset "imc2024_dioscuri_baalshamin" -> num_images=138
Dataset "imc2024_lizard_pond" -> num_images=214
Dataset "pt_brandenburg_british_buckingham" -> num_images=225
Dataset "pt_piazzasanmarco_grandplace" -> num_images=168
Dataset "pt_sacrecoeur_trevi_tajmahal" -> num_images=225
Dataset "pt_stpeters_stpauls" -> num_images=200
Dataset "amy_gardens" -> num_images=200
Dataset "fbk_vineyard" -> num_images=163
Dataset "ETs" -> num_images=22
Dataset "stairs" -> num_images=51


In [15]:

gc.collect()

max_images = None  # Used For debugging only. Set to None to disable.
datasets_to_process = None  # Not the best convention, but None means all datasets.

if is_train:
    # max_images = 5

    # Note: When running on the training dataset, the notebook will hit the time limit and die. Use this filter to run on a few specific datasets.
    datasets_to_process = [
    	# New data.
    	# 'amy_gardens',
    	# 'ETs',
    	# 'fbk_vineyard',
    	'stairs', 
    	# Data from IMC 2023 and 2024.
    	# 'imc2024_dioscuri_baalshamin',
    	# 'imc2023_theather_imc2024_church',
    	# 'imc2023_heritage',
    	# 'imc2023_haiper',
    	# 'imc2024_lizard_pond',
    	# Crowdsourced PhotoTourism data.
    	# 'pt_stpeters_stpauls',
    	# 'pt_brandenburg_british_buckingham',
    	# 'pt_piazzasanmarco_grandplace',
    	# 'pt_sacrecoeur_trevi_tajmahal',
    ]

timings = {
    "shortlisting":[],
    "feature_detection": [],
    "feature_matching":[],
    "image doppelgangers processing":[],
    "RANSAC": [],
    "Reconstruction": [],
}
mapping_result_strs = []


print (f"Extracting on device {device}")
for dataset, predictions in samples.items():
    if datasets_to_process and dataset not in datasets_to_process:
        print(f'Skipping "{dataset}"')
        continue
    
    images_dir = os.path.join(data_dir, 'train' if is_train else 'test', dataset)
    images = [os.path.join(images_dir, p.filename) for p in predictions]
    if max_images is not None:
        images = images[:max_images]

    print(f'\nProcessing dataset "{dataset}": {len(images)} images')

    filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}

    feature_dir = os.path.join(workdir, 'featureout', dataset)
    os.makedirs(feature_dir, exist_ok=True)

    # Wrap algos in try-except blocks so we can populate a submission even if one scene crashes.
    try:
        t = time()
        index_pairs = get_image_pairs_shortlist(
            images,
            sim_th = 0.5, # should be strict
            min_pairs = 10, # we should select at least min_pairs PER IMAGE with biggest similarity
            exhaustive_if_less = 20,
            device=device
        )
        timings['shortlisting'].append(time() - t)
        print (f'Shortlisting. Number of pairs to match: {len(index_pairs)}. Done in {time() - t:.4f} sec')
        gc.collect()
    
        t = time()

        detect_aliked(images, feature_dir, 8192, device=device)
        gc.collect()
        timings['feature_detection'].append(time() - t)
        print(f'Features detected in {time() - t:.4f} sec')

        t = time()
        pair_probability_file_path = exec_doppelgangers_classifier(
            Path(images_dir),
            images,
            Path(feature_dir),
            index_pairs,
            Config.num_gpus,
            **Config.doppelgangers_classifier_args,
        )
        torch.cuda.empty_cache()
        gc.collect()
        updated_index_pairs = extract_matching_list_from_h5(pair_probability_file_path, filename_to_index)
        print(f"updates matches with LOFTR from {len(index_pairs)} to {len(updated_index_pairs)}")
        timings['image doppelgangers processing'].append(time() - t)
        print(f'Features matched in {time() - t:.4f} sec')

        t = time()
        match_with_lightglue(images, updated_index_pairs, feature_dir=feature_dir, device=device, verbose=False)
        # match_with_lightglue_and_cluster(images, index_pairs, feature_dir=feature_dir, device=device, verbose=False)
        timings['feature_matching'].append(time() - t)
        print(f'Features matched in {time() - t:.4f} sec')

        database_path = os.path.join(feature_dir, 'colmap.db')
        if os.path.isfile(database_path):
            os.remove(database_path)
        gc.collect()
        sleep(1)
        import_into_colmap(images_dir, feature_dir=feature_dir, database_path=database_path)
        output_path = f'{feature_dir}/colmap_rec_aliked'
        
        t = time()
        pycolmap.match_exhaustive(database_path)
        timings['RANSAC'].append(time() - t)
        print(f'Ran RANSAC in {time() - t:.4f} sec')
        
        # By default colmap does not generate a reconstruction if less than 10 images are registered.
        # Lower it to 3.
        mapper_options = pycolmap.IncrementalPipelineOptions()
        mapper_options.min_model_size = 5
        mapper_options.max_num_models = 25
        mapper_options.mapper.filter_max_reproj_error	 = 6.0
        # mapper_options.min_num_matches	 = 50
        # mapper_options.ba_local_max_num_iterations = 100
        # mapper_options.ba_local_num_images = 10
        mapper_options.ba_global_images_freq = 5
        

        os.makedirs(output_path, exist_ok=True)
        t = time()
        maps = pycolmap.incremental_mapping(
            database_path=database_path, 
            image_path=images_dir,
            output_path=output_path,
            options=mapper_options)
        sleep(1)
        timings['Reconstruction'].append(time() - t)
        print(f'Reconstruction done in  {time() - t:.4f} sec')
        print(maps)

        # clear_output(wait=False)
    
        registered = 0
        for map_index, cur_map in maps.items():
            img_list =[]
            for index, image in cur_map.images.items():
                prediction_index = filename_to_index[image.name]
                predictions[prediction_index].cluster_index = map_index
                predictions[prediction_index].rotation = deepcopy(image.cam_from_world.rotation.matrix())
                predictions[prediction_index].translation = deepcopy(image.cam_from_world.translation)
                img_list.append(image.name)
                registered += 1
            img_list_str = ' '.join(img_list) 
            print(f"map_index = {map_index}", img_list_str)
        mapping_result_str = f'Dataset "{dataset}" -> Registered {registered} / {len(images)} images with {len(maps)} clusters'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)
        gc.collect()
    except Exception as e:
        print(e)
        # raise e
        mapping_result_str = f'Dataset "{dataset}" -> Failed!'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)

print('\nResults')
for s in mapping_result_strs:
    print(s)

print('\nTimings')
for k, v in timings.items():
    print(f'{k} -> total={sum(v):.02f} sec.')

Extracting on device cpu
Skipping "imc2023_haiper"
Skipping "imc2023_heritage"
Skipping "imc2023_theather_imc2024_church"
Skipping "imc2024_dioscuri_baalshamin"
Skipping "imc2024_lizard_pond"
Skipping "pt_brandenburg_british_buckingham"
Skipping "pt_piazzasanmarco_grandplace"
Skipping "pt_sacrecoeur_trevi_tajmahal"
Skipping "pt_stpeters_stpauls"
Skipping "amy_gardens"
Skipping "fbk_vineyard"
Skipping "ETs"

Processing dataset "stairs": 51 images


100%|██████████| 51/51 [00:34<00:00,  1.49it/s]


Distance Matrix Statistics:
Min:  0.1598
Max:  0.4240
Mean: 0.2807
Std:  0.0452
20%:  0.2433
30%:  0.2557
USED 50%:  0.2767
75%:  0.3089
Shortlisting. Number of pairs to match: 647. Done in 36.5036 sec


100%|██████████| 51/51 [12:19<00:00, 14.50s/it]


Features detected in 740.3003 sec
/kaggle/working/result/featureout/stairs
number sections must be larger than 0.
Dataset "stairs" -> Failed!

Results
Dataset "stairs" -> Failed!

Timings
shortlisting -> total=36.50 sec.
feature_detection -> total=740.30 sec.
feature_matching -> total=0.00 sec.
image doppelgangers processing -> total=0.00 sec.
RANSAC -> total=0.00 sec.
Reconstruction -> total=0.00 sec.


In [16]:
# Must Create a submission file.

array_to_str = lambda array: ';'.join([f"{x:.09f}" for x in array])
none_to_str = lambda n: ';'.join(['nan'] * n)

submission_file = '/kaggle/working/submission.csv'
with open(submission_file, 'w') as f:
    if is_train:
        f.write('dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')
    else:
        f.write('image_id,dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.image_id},{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')

!head {submission_file}

dataset,scene,image,rotation_matrix,translation_vector
imc2023_haiper,outliers,fountain_image_116.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_108.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_101.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_082.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_071.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_025.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_000.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_007.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_012.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan


In [17]:
# Definitely Compute results if running on the training set.
# Do not do this when submitting a notebook for scoring. All you have to do is save your submission to /kaggle/working/submission.csv.

if is_train:
    t = time()
    final_score, dataset_scores = metric.score(
        gt_csv='/kaggle/input/image-matching-challenge-2025/train_labels.csv',
        user_csv=submission_file,
        thresholds_csv='/kaggle/input/image-matching-challenge-2025/train_thresholds.csv',
        mask_csv=None if is_train else os.path.join(data_dir, 'mask.csv'),
        inl_cf=0,
        strict_cf=-1,
        verbose=True,
    )
    print(f'Computed metric in: {time() - t:.02f} sec.')

imc2023_haiper: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2023_heritage: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2023_theather_imc2024_church: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2024_dioscuri_baalshamin: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2024_lizard_pond: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_brandenburg_british_buckingham: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_piazzasanmarco_grandplace: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_sacrecoeur_trevi_tajmahal: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_stpeters_stpauls: score=0.00% (mAA=0.00%, clusterness=0.00%)
amy_gardens: score=0.00% (mAA=0.00%, clusterness=0.00%)
fbk_vineyard: score=0.00% (mAA=0.00%, clusterness=0.00%)
ETs: score=0.00% (mAA=0.00%, clusterness=0.00%)
stairs: score=0.00% (mAA=0.00%, clusterness=0.00%)
Average over all datasets: score=0.00% (mAA=0.00%, clusterness=0.00%)
Computed metric in: 0.08 sec.
